# Setup

In [1]:
import os, base64
from dotenv import load_dotenv
from openai import OpenAI
from pdf2image import convert_from_path

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from IPython.display import Markdown, display

# Perform OCR and transform to images

In [3]:
def pdf_to_images(pdf_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    imagens = convert_from_path(pdf_path)
    image_paths = []

    pdf_nome = os.path.splitext(os.path.basename(pdf_path))[0]
    for i, imagem in enumerate(imagens):
        image_path = os.path.join(output_folder, f"{pdf_nome}_pg{i+1}.jpg")
        imagem.save(image_path, "JPEG")
        image_paths.append(image_path)

    return image_paths

In [4]:
# Convertendo pdf para img
main_folder = os.path.join(os.getcwd(), "5.rag_openai")
receips_folder = os.path.join(main_folder, "receips")
output_folder = os.path.join(main_folder, "img")

pdf_path = os.path.join(receips_folder, "Southern Cookbook of Fine Recipes.pdf")
image_paths = pdf_to_images(pdf_path, output_folder)

In [5]:
client = OpenAI(api_key=openai_api_key)
model = "gpt-4o-mini"

In [7]:
# Read and encode one image
image_path = os.path.join(output_folder, "Southern Cookbook of Fine Recipes_pg9.jpg")
with open(image_path, "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')

In [8]:
system_prompt = """
Analyse the content of this image and extract any related recipe information.
"""

In [14]:
def generate(client, system_prompt, image, model="gpt-4o-mini"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": [
                "This is the image from the recipe page.",
                {"type": "image_url",
                 "image_url": {"url": f"data:image/jpeg;base64,{image}",
                               "detail": "high"}}
            ]}
        ]
    )
    return response.choices[0].message.content

In [15]:
response = generate(client, system_prompt, image_data)
display(Markdown(response))

Here's a summary of the recipes included in the provided image:

### Chicken Gumbo
- **Ingredients**: 
  - 1 small stewing chicken
  - 2 tablespoons flour
  - 3 tablespoons butter, melted
  - 1 onion, chopped
  - 4 cups okra, sliced and chopped
  - 2 cups tomato pulp
  - Few sprigs parsley, chopped
  - Salt and pepper to taste
- **Instructions**: 
  1. Clean and cut chicken into portions.
  2. Dredge chicken in flour.
  3. Sauté in melted butter with chopped onion until browned.
  4. Season with salt and pepper.
  5. Cook slowly for about 2½ hours until chicken is tender.
  6. Add okra and parsley, cooking until okra is well done.

### Almond Chicken Soup
- **Ingredients**: 
  - ½ cup blanched almonds
  - 6 bitter almonds
  - 3 cups chicken broth
  - 1 teaspoon onion juice
  - 1 bay leaf, crushed fine
  - 3 tablespoons butter
  - 2 cups milk
  - 1 cup cream
  - Salt and pepper to taste
- **Instructions**: 
  1. Chop almonds and add to broth with seasoning.
  2. Melt butter, stir in flour, add to broth until smooth.
  3. Add milk and cream and season with salt and pepper.

### Southern Bean Soup
- **Ingredients**: 
  - 1 cup dried beans
  - 6 cups ham broth
  - 1 cup chopped celery
  - ½ onion, minced
  - 3 tablespoons butter
  - 3 tablespoons flour
  - Salt and pepper
- **Instructions**: 
  1. Cover beans with water and let stand overnight.
  2. Drain and add to ham broth.
  3. Cook with celery and onion until beans are soft.
  4. Strain broth, press beans through a sieve, and add to broth.
  5. Thicken with butter, flour, salt, and pepper.

### Additional Recipes
- **Chicken Cream Soup**
- **Okra Soup**
- **Black Bean Soup**
- **Southern Gumbo**

These recipes reflect traditional Southern cooking, featuring hearty ingredients and methods that emphasize slow cooking and robust flavors.